# A. State preparation with Jaynes-Cummings controls

In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.grape_paramaterized import optimize_pulse_parameterized
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

As a preliminary step, we consider state preparation
of a target state starting from a pure state. In addition,
we assume that any coupling to an external environment
is negligible and that the parametrized controls can be
implemented perfectly.

Here no feedback is required, we are just testing the parameterized gates setup.

As a first example, we consider the state preparation
of a cavity resonantly coupled to an externally driven
qubit

Here, we consider a particular sequence of
parametrized unitary gates originally introduced by Law
and Eberly

In [2]:
N_cav = 10

In [3]:
def qubit_unitary(alpha_re, alpha_im):
    """
    TODO: see if alpha, can be sth elser other than scalar, and if the algo understands this
    see if there can be multiple params like alpha and beta input
    """
    alpha = alpha_re + 1j * alpha_im
    return tensor(identity(N_cav),  expm(
        -1j
        * (
            alpha * sigmap()
            + alpha.conjugate() * sigmam()
        )
        / 2
    ))

In [4]:
def qubit_cavity_unitary(beta_re, beta_im):
    beta = beta_re + 1j * beta_im
    return expm(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), sigmap())
            )
            + beta.conjugate()
            * (
                tensor(create(N_cav), sigmam())
            )
        )
        / 2
    )

In [5]:
Uq = qubit_unitary(0.1, 0.1)
Uqc = qubit_cavity_unitary(0.2, 0.2)
print(
    "Uq unitary check:",
    jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
)
print(
    "Uqc unitary check:",
    jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
)

Uq unitary check: True
Uqc unitary check: True


In [6]:
qubit_unitary(0.1, 0.1).shape

(20, 20)

In [7]:
qubit_unitary(0.1, 0.1)

Array([[ 0.99750104+6.19847336e-21j,  0.04995834-4.99583437e-02j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [-0.04995834-4.99583437e-02j,  0.99750104-2.44545153e-20j,
         0.        -0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        -0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        -0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        -0.00000000e+00j,  0.        +0.00000000e+00j,
         

In their groundbreaking work, Law and Eberly have
shown that any arbitrary superposition of Fock states with
maximal excitation number N can be prepared out of the
ground state in a sequence of N such interleaved gates,
also providing an algorithm to find the correct angles and
interaction durations

In [8]:
print(qubit_cavity_unitary(0.1, 0.1).shape)

(20, 20)


In [9]:
qubit_cavity_unitary(0.1, 0.1)

Array([[ 0.99750104+6.19847336e-21j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.04995834-4.99583437e-02j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.        +0.00000000e+00j,  1.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         

## First target is the state $ | 1 ⟩ + | 3 ⟩ $ 

In [10]:
time_steps = 3  # corressponds to maximal excitation number of an arbitrary Fock State Superposition

In [11]:
psi0 = tensor(basis(N_cav), basis(2))
psi0 = psi0 / jnp.linalg.norm(psi0)
psi_target = tensor((fock(N_cav, 1) + fock(N_cav, 3)), basis(2))
psi_target = psi_target / jnp.linalg.norm(psi_target)

In [12]:
psi0.shape

(20, 1)

In [13]:
psi_target.shape

(20, 1)

In [14]:
print(fock(N_cav, 1))

[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


Law and Eberly provided an algorithm to determine the correct parameters for state preparation. These include:

- The rotation angle $ |\alpha| $,
- The azimuthal angle $ \arg\left(\frac{\alpha}{|\alpha|}\right) $,
- The interaction duration $ |\beta| $. <br>

So Goal is to find the best control vector (rather than control amplitudes, this time) that leads to finding the optimal state-preparation strategies. Performing as well as the Law-Eberly algorithm.

In [ ]:
# TODO: see why if initial [paramaters are complex, adam fidelity is so bad and l-bfgs does error]
import jax.random as random

key = random.PRNGKey(0)  # Random seed for reproducibility
num_gates = len(
    [qubit_unitary, qubit_cavity_unitary]
)  # Number of parameterized gates
# doing a .tolist() will lead to weird shape if you multiply outside with 5 or sth
# playing wiht the range will increase or decrease fidelity
initial_parameters = random.uniform(
    key, shape=(time_steps, num_gates, 2), minval=-5, maxval=5
)
print(f"Initial parameters: \n {initial_parameters}")

Initial parameters: 
 [[[-0.81542888 -2.83704545]
  [ 4.65321461  0.74500534]]

 [[ 0.32226488 -1.4509482 ]
  [ 3.83008976  1.3256039 ]]

 [[ 0.33800077 -3.08346119]
  [ 3.54176652 -4.40560709]]]


## Optimizing
Currently l-bfgs with the same learning rate of 0.3 converges at a local minimum of 0.5, adam also converges at 0.5 but at smaller learning rates

In [16]:
result = optimize_pulse_parameterized(
    U_0=psi0,
    C_target=psi_target,
    parameterized_gates=[qubit_unitary, qubit_cavity_unitary],
    initial_parameters=initial_parameters,
    num_time_steps=time_steps,
    optimizer="adam",
    max_iter=1000,
    convergence_threshold=1e-6,
    type="state",
    propcomp="time-efficient",
    learning_rate=0.01,
)

In [17]:
jnp.array(result.control_amplitudes)

Array([[[-0.79503762, -3.04102924],
        [ 3.67923377, -0.59932759]],

       [[ 2.25773434, -2.1746021 ],
        [ 3.07182014, -1.11438757]],

       [[ 0.5378215 , -3.10031286],
        [ 2.06797918, -5.03315999]]], dtype=float64)

In [18]:
print(result)

result(control_amplitudes=Array([[[-0.79503762, -3.04102924],
        [ 3.67923377, -0.59932759]],

       [[ 2.25773434, -2.1746021 ],
        [ 3.07182014, -1.11438757]],

       [[ 0.5378215 , -3.10031286],
        [ 2.06797918, -5.03315999]]], dtype=float64), final_fidelity=Array(0.99998835, dtype=float64), iterations=299, final_operator=Array([[ 3.84475986e-04+9.77963261e-04j],
       [-4.83289251e-04+4.07094615e-04j],
       [-4.69985179e-01-5.28329607e-01j],
       [ 6.43953080e-04+1.12371571e-03j],
       [ 1.10228014e-03-4.94033640e-04j],
       [-1.25782650e-05-2.01852819e-05j],
       [-4.72035081e-01-5.26456879e-01j],
       [-1.39413985e-03-1.10017267e-03j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.

In [19]:
from feedback_grape.grape_paramaterized import calculate_trajectory
from feedback_grape.utils.fidelity import fidelity

U_final = calculate_trajectory(
    psi0,
    result.control_amplitudes,
    time_steps,
    [qubit_unitary, qubit_cavity_unitary],
    type="state",
)
print(fidelity(C_target=psi_target, U_final=U_final, type="state"))

0.9999883507922764
